In [7]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import statsmodels.api as sm
import math
import psycopg2
import datetime

today = datetime.date.today()
day = today.strftime('%d')
#startMonth = today.strftime('%m')
startMonth = '0' + str(int(today.strftime('%m')) - 1)
endMonth = '0' + str(int(startMonth) + 2)
year = today.year

startDate = str(year)+'-'+str(startMonth)+'-'+'01'+' '+'00:00:00'
endDate = str(year)+'-'+str(endMonth)+'-'+'01'+' '+'00:00:00'

#Importing data
df = pd.read_csv('2018_hourly_data_10X_1.csv')   #2018_hourly_data.csv #1year_hourly_data_2015.csv
# df.describe()

#Creating train and test set 
train=df[0:8784] #80% of data ( 0.8 * 8755 for training the model) 
test=df[8784:] # remaining 20% to test

#Aggregating the dataset at daily level
df.Timestamp = pd.to_datetime(df.dateformat_hour,format='%Y-%m-%d %H:%M:%S') 
df.index = df.Timestamp 
df = df.resample('H').mean() #daily mean
train.Timestamp = pd.to_datetime(train.dateformat_hour,format='%Y-%m-%d %H:%M:%S') 
train.index = train.Timestamp 
train = train.resample('H').mean() 
test.Timestamp = pd.to_datetime(test.dateformat_hour,format='%Y-%m-%d %H:%M:%S') 
test.index = test.Timestamp 
test = test.resample('H').mean()

y_hat_avg = test.copy()
fit1 = sm.tsa.SARIMAX(train, order=(3, 1, 3),seasonal_order=(0,1,1,24)).fit()
y_hat_avg['SARIMA'] = predictions = fit1.predict(start=startDate, end=endDate, dynamic=True)

predictedValues = list()
for t in predictions:
    predictedValues.append(math.ceil(t))

print(predictedValues)




conn = psycopg2.connect("dbname=merakidb user=postgres password=postgres")
cur = conn.cursor()

yearParam = today.year
monthparam = today.month -1

x = datetime.datetime(yearParam,monthparam,1)
count = 0

cur.execute("DELETE FROM meraki.prediction_value_table")
for i in range(len(predictions)):
    
    if(count == 24):
        x += datetime.timedelta(days=1)
        count=0
    else:
        time = x.strftime("%x")
        visitorCount = predictedValues[i]
        cur.execute("INSERT INTO meraki.prediction_value_table (dateformat_date,count) VALUES (%s, %s);", (time, visitorCount))
        conn.commit()
        print(time,visitorCount)
    
    count += 1

C:\Users\sonia_khatri\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:28: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
C:\Users\sonia_khatri\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:31: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
C:\Users\sonia_khatri\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:34: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
C:\Users\sonia_khatri\AppData\Roaming\Python\Python37\site-packages\statsmodels\base\model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarnin

[-50, -52, -54, -57, -59, -56, -61, -61, 468, 525, 497, 477, 509, 482, 503, 469, 483, 473, 880, 887, 474, 461, 484, -50, -52, -53, -55, -58, -60, -57, -62, -62, 467, 523, 496, 476, 508, 480, 502, 468, 482, 472, 879, 886, 473, 460, 483, -51, -53, -54, -56, -59, -61, -58, -63, -63, 466, 522, 495, 475, 507, 479, 501, 467, 481, 471, 877, 885, 472, 459, 482, -52, -54, -55, -57, -60, -62, -59, -64, -64, 465, 521, 494, 474, 506, 478, 500, 466, 480, 470, 876, 884, 471, 458, 480, -53, -55, -56, -58, -61, -63, -60, -66, -65, 464, 520, 493, 473, 505, 477, 499, 465, 479, 469, 875, 883, 469, 457, 479, -54, -56, -57, -59, -62, -64, -62, -67, -66, 463, 519, 491, 472, 504, 476, 498, 464, 478, 468, 874, 882, 468, 456, 478, -55, -57, -59, -60, -63, -65, -63, -68, -67, 462, 518, 490, 471, 503, 475, 497, 462, 477, 467, 873, 881, 467, 455, 477, -56, -58, -60, -61, -64, -66, -64, -69, -68, 461, 517, 489, 470, 501, 474, 495, 461, 476, 465, 872, 880, 466, 454, 476, -57, -59, -61, -62, -65, -67, -65, -70, -70,

05/04/19 480
05/04/19 -53
05/05/19 -56
05/05/19 -58
05/05/19 -61
05/05/19 -63
05/05/19 -60
05/05/19 -66
05/05/19 -65
05/05/19 464
05/05/19 520
05/05/19 493
05/05/19 473
05/05/19 505
05/05/19 477
05/05/19 499
05/05/19 465
05/05/19 479
05/05/19 469
05/05/19 875
05/05/19 883
05/05/19 469
05/05/19 457
05/05/19 479
05/05/19 -54
05/06/19 -57
05/06/19 -59
05/06/19 -62
05/06/19 -64
05/06/19 -62
05/06/19 -67
05/06/19 -66
05/06/19 463
05/06/19 519
05/06/19 491
05/06/19 472
05/06/19 504
05/06/19 476
05/06/19 498
05/06/19 464
05/06/19 478
05/06/19 468
05/06/19 874
05/06/19 882
05/06/19 468
05/06/19 456
05/06/19 478
05/06/19 -55
05/07/19 -59
05/07/19 -60
05/07/19 -63
05/07/19 -65
05/07/19 -63
05/07/19 -68
05/07/19 -67
05/07/19 462
05/07/19 518
05/07/19 490
05/07/19 471
05/07/19 503
05/07/19 475
05/07/19 497
05/07/19 462
05/07/19 477
05/07/19 467
05/07/19 873
05/07/19 881
05/07/19 467
05/07/19 455
05/07/19 477
05/07/19 -56
05/08/19 -60
05/08/19 -61
05/08/19 -64
05/08/19 -66
05/08/19 -64
05/08/19 -69

06/01/19 853
06/01/19 440
06/01/19 427
06/01/19 450
06/01/19 -84
06/02/19 -87
06/02/19 -89
06/02/19 -92
06/02/19 -94
06/02/19 -91
06/02/19 -96
06/02/19 -96
06/02/19 433
06/02/19 489
06/02/19 462
06/02/19 442
06/02/19 474
06/02/19 446
06/02/19 468
06/02/19 434
06/02/19 448
06/02/19 438
06/02/19 845
06/02/19 852
06/02/19 439
06/02/19 426
06/02/19 449
06/02/19 -85
06/03/19 -88
06/03/19 -90
06/03/19 -93
06/03/19 -95
06/03/19 -92
06/03/19 -97
06/03/19 -97
06/03/19 432
06/03/19 488
06/03/19 461
06/03/19 441
06/03/19 473
06/03/19 445
06/03/19 467
06/03/19 433
06/03/19 447
06/03/19 437
06/03/19 843
06/03/19 851
06/03/19 438
06/03/19 425
06/03/19 447
06/03/19 -86
06/04/19 -89
06/04/19 -91
06/04/19 -94
06/04/19 -96
06/04/19 -93
06/04/19 -98
06/04/19 -98
06/04/19 431
06/04/19 487
06/04/19 460
06/04/19 440
06/04/19 472
06/04/19 444
06/04/19 466
06/04/19 432
06/04/19 446
06/04/19 436
06/04/19 842
06/04/19 850
06/04/19 437
06/04/19 424
06/04/19 446
06/04/19 -87
06/05/19 -90
06/05/19 -92
06/05/19 -95

In [8]:
/

05/01/19 9
05/01/19 9
05/01/19 9
05/01/19 9
05/01/19 9
05/01/19 9
05/01/19 9
05/01/19 9
05/01/19 123
05/01/19 124
05/01/19 124
05/01/19 125
05/01/19 123
05/01/19 123
05/01/19 123
05/01/19 124
05/01/19 123
05/01/19 123
05/01/19 164
05/01/19 163
05/01/19 124
05/01/19 124
05/01/19 124
05/01/19 9
05/02/19 9
05/02/19 9
05/02/19 9
05/02/19 9
05/02/19 9
05/02/19 9
05/02/19 9
05/02/19 123
05/02/19 124
05/02/19 124
05/02/19 125
05/02/19 123
05/02/19 123
05/02/19 123
05/02/19 124
05/02/19 123
05/02/19 123
05/02/19 164
05/02/19 163
05/02/19 124
05/02/19 124
05/02/19 124
05/02/19 9
05/03/19 9
05/03/19 9
05/03/19 9
05/03/19 9
05/03/19 9
05/03/19 9
05/03/19 9
05/03/19 123
05/03/19 124
05/03/19 124
05/03/19 125
05/03/19 123
05/03/19 123
05/03/19 123
05/03/19 124
05/03/19 123
05/03/19 123
05/03/19 164
05/03/19 164
05/03/19 124
05/03/19 124
05/03/19 124
05/03/19 9
05/04/19 9
05/04/19 9
05/04/19 9
05/04/19 9
05/04/19 9
05/04/19 9
05/04/19 9
05/04/19 123
05/04/19 124
05/04/19 124
05/04/19 125
05/04/19 12

In [42]:
import datetime
today = datetime.date.today()
day = today.strftime('%d')
startMonth = today.strftime('%m')
endMonth = '0' + str(int(startMonth) + 1)
year = today.year

startDate = str(year)+'-'+str(startMonth)+'-'+'01'+' '+'00:00:00'
endDate = str(year)+'-'+str(endMonth)+'-'+'01'+' '+'00:00:00'

print(today.day)

28


In [6]:
import datetime

today = datetime.date.today()
day = today.strftime('%d')
#startMonth = today.strftime('%m')
startMonth = '0' + str(int(today.strftime('%m')) - 2)
endMonth = '0' + str(int(startMonth) + 2)
year = today.year
print(today.month -1)

5
